In [1]:
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SQLContext, SparkSession
import pyspark.sql.functions as sf

from RecMet import recmet

In [2]:
spark = SparkSession.builder.getOrCreate()


22/03/02 02:30:00 WARN Utils: Your hostname, machine resolves to a loopback address: 127.0.1.1; using 192.168.31.230 instead (on interface enp0s3)
22/03/02 02:30:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 02:30:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 02:30:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
cols= ['User','Recommended','Test']
data = [[1,['prod_1','prod_2','prod_3','prod_4','prod_5'],['prod_10','prod_4','prod_5']],
        [2,['prod_10','prod_12','prod_3','prod_4','prod_15'],['prod_1','prod_4','prod_15']],
        [3,['prod_11','prod_7','prod_8','prod_9','prod_15'],['prod_1','prod_2','prod_3','prod_7','prod_9','prod_11','prod_10','prod_4','prod_15']],
        [4,['prod_9','prod_12','prod_13','prod_14','prod_15'],['prod_12']]]

pdf_rec = pd.DataFrame(data, columns=cols)
# pdf_rec.to_csv('./test_data.csv')
# pdf_rec = pd.read_csv('./test_data.csv')
sdf_rec = spark.createDataFrame(pdf_rec)
sdf_rec = sdf_rec.withColumn('TestLen',sf.size(sf.col('Test')))
print(sdf_rec.dtypes)
sdf_rec.show()



[('User', 'bigint'), ('Recommended', 'array<string>'), ('Test', 'array<string>'), ('TestLen', 'int')]


+----+--------------------+--------------------+-------+
|User|         Recommended|                Test|TestLen|
+----+--------------------+--------------------+-------+
|   1|[prod_1, prod_2, ...|[prod_10, prod_4,...|      3|
|   2|[prod_10, prod_12...|[prod_1, prod_4, ...|      3|
|   3|[prod_11, prod_7,...|[prod_1, prod_2, ...|      9|
|   4|[prod_9, prod_12,...|           [prod_12]|      1|
+----+--------------------+--------------------+-------+



In [4]:
# Randomly Initialise variables
weight_dict ={
    'prod_1':1,
    'prod_2':2,
    'prod_3':3,
    'prod_4':4,
    'prod_5':5,
    'prod_6':6,
    'prod_7':7,
    'prod_8':8,
    'prod_9':9,
    'prod_10':10,
    'prod_11':11,
    'prod_12':12,
    'prod_13':13,
    'prod_14':14,
    'prod_15':15,
    'prod_16':16,
    'prod_17':17,
    'prod_18':18,
}
weights =  weight_dict
longtail_thresh = 8


In [5]:

rm = recmet(weights,longtail_thresh)
sdf_rec_metric= sdf_rec.withColumn('AI',rm.AI(sf.col('Recommended'), sf.col('Test')))\
    .withColumn('PWAI',rm.PWAI(sf.col('Recommended'), sf.col('Test')))\
    .withColumn('ARP',rm.ARP(sf.col('Recommended')))\
    .withColumn('APLT',rm.APLT(sf.col('Recommended')))\
    .withColumn('ACLT',rm.ACLT(sf.col('Recommended')))
sdf_rec_metric.show()

+----+--------------------+--------------------+-------+------+------+----+----+----+
|User|         Recommended|                Test|TestLen|    AI|  PWAI| ARP|APLT|ACLT|
+----+--------------------+--------------------+-------+------+------+----+----+----+
|   1|[prod_1, prod_2, ...|[prod_10, prod_4,...|      3|0.6667|0.4737| 3.0| 0.0| 0.0|
|   2|[prod_10, prod_12...|[prod_1, prod_4, ...|      3|0.6667|  0.95| 8.8| 0.6| 3.0|
|   3|[prod_11, prod_7,...|[prod_1, prod_2, ...|      9|   0.8|  0.84|10.0| 0.6| 3.0|
|   4|[prod_9, prod_12,...|           [prod_12]|      1|   1.0|   1.0|12.6| 1.0| 5.0|
+----+--------------------+--------------------+-------+------+------+----+----+----+

